In [1]:
import pandas as pd
import numpy as np

### Для кластеризации магазинов по статичным (или почти статичным) параметрам будем использовать алгоритм K-Means. 

1. Поскольку много признаков магазинов являются категориальными (в основном бинарными, как dummy-регионы, изменение юр лица, вид магазина (ТЦ или отдельный)) и предикторов много, а объектов мало, то для начала выполняется PCA (принцип главных компонент) для уменьшения размерности пространства предикторов (т.е. из исходных N предикторов  составляются новые (лин комбинации) M предикторов, первые из которых лучше всего описывают поведение множества).
2. После этого берутся первые полученные предикторы (в нашем случае первые 10) и составляется новая таблица "объект-признак". Далее, проводится приведение признаков к единой шкале (стандартизация) для устранения эффекта смещения кластеров. После этого мы имеем итоговую таблицу для кластеризации.
3. Проводим кластеризацию, разбивая на 4 кластера (взяли 4, т. к. меньше -- совсем мало, а больше -- не уверены, что нужно, т.к. иначе появится более 4 новых предикторов для итогового прогноза)

In [20]:
from sklearn.decomposition import PCA # импортируем необходимые алгоритмы
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [21]:
#считываем итоговую таблицу по магазинам
df = pd.read_csv("C:\\Users\\Pavel.Klyuchevskiy\\Desktop\\mine\\RED\\CITY.csv", sep=';',encoding='1251')
df3=df[['Name','SHOP']]

In [22]:
#удаляем признаки, которые являются побочными (например, признак GROUP отвечает за регион магазина и уже разбит в таблице на dummy)
df.drop(['GROUP','Name','SHOP','OPEN','CLOSE','PLACE','CHANGE_DATE','CHANGE_DATE_1','CLOSE_1'],inplace=True,axis=1)

In [26]:
#создаем PCA с 8 главными компонентами
pca=PCA(n_components=8,random_state=0)

In [27]:
#обучаем PCA (таблица взята до предпоследнего наблюдения, т.к. по Сергиеву Посаду не было информации)
df1 = pca.fit_transform(df)

In [28]:
#Здесь выводим то, насколько каждый из 10 новых полученных предикторов объясняет поведение множества магазинов)
#Чем больше номер признака, тем меньше он объясняет поведение множества магазинов. Как видим, первый образованный признак объясняет большую часть.
print(pca.explained_variance_ratio_)

[0.76739826 0.08258053 0.06072387 0.0288805  0.02448109 0.01963087
 0.00575291 0.00371944]


In [29]:
# для себя вывели сингулярные значения матрицы 
print(pca.singular_values_)

[1929701.71817134  633021.7267652   542824.57704596  374353.69141821
  344663.35459065  308638.1453741   167079.62375796  134344.15512579]


In [30]:
#конвертируем полученные 10 прзинаков в датафрейм
df2 = pd.DataFrame(df1)

In [31]:
#стандартизируем таблицу
df4 = StandardScaler().fit_transform(df2)

In [32]:
#создаем алгоритм K-Means с 4 кластерами и обучаем на стандартизированных данных
kmeans = KMeans(n_clusters=4, random_state=0).fit(df4)

In [34]:
#выводим полученные кластеры
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 3, 3, 1, 2, 2, 1, 2, 2,
       3, 3, 3, 0, 0, 3, 2, 3, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0,
       0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3,
       1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 2])

In [35]:
#конкатенируем название магазина и присвоенный кластер
b1 = pd.DataFrame(kmeans.labels_)
b1.columns = ['Pred']
a1 = pd.concat([df3['SHOP'],pd.DataFrame(df4),b1],axis=1)

In [36]:
#группируем по кластеру. считая число магазинов, попавших в каждый кластер
a1.groupby('Pred')['SHOP'].count()

Pred
0    20
1     7
2    57
3     9
Name: SHOP, dtype: int64

In [37]:
#вывод итоговой таблицы
a1[['SHOP','Pred']]

,SHOP,Pred
0,12NG1007,2
1,12NG1012,2
2,12NG1039,2
3,12NG1040,2
4,12NG1049,2
5,12NG1050,2
6,12NG1068,2
7,12TA1005,0
8,12TA1006,0
9,12TA1008,2


In [38]:
a1[['SHOP','Pred']].to_csv('Clusters1.csv')